In [1]:
from __future__ import print_function
import os
import sys
PROJ_ROOT = os.path.join(os.pardir)
print(os.path.abspath(PROJ_ROOT))
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)

/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/runnable_program


In [2]:
# Data manipulation
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
import asyncio
import requests

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython import get_ipython
ipython = get_ipython()
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 1
# Use %aimport module to reload each module

# Visualizations
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
import glob
raw_data_folder = os.path.join(PROJ_ROOT,
                                "data",
                                "raw")
raw_files = glob.glob(raw_data_folder + "/*.csv")
refined_data_folder = os.path.join(PROJ_ROOT,
                                "data",
                                "interim")
refined_files = glob.glob(refined_data_folder + "/*.csv")
sample_df = pd.read_csv(raw_files[0],
                        index_col=['site_id', 'time'], parse_dates=True)
sample_df = sample_df[(sample_df.index.get_level_values(0) != 16)]
sample_df = sample_df[['CO', 'NO2', 'PM25']]

In [7]:
# Read refined files and interpolate all 0 in them with slinear method
import pandas as pd
import numpy as np
import glob
import xarray as xr

idx = pd.IndexSlice

refined_df = pd.DataFrame()
new_refined_df = pd.DataFrame()
for file in refined_files:
    print('Currently processing file \n{}'.format(file))
    refined_df = refined_df.append(pd.read_csv(file, parse_dates=True, index_col=['site_id', 'time']))
    
refined_df = refined_df.drop(index=[16,49,48], level=0)
refined_df = refined_df.drop(columns=['NowCast','Continous length'])
site_ids = refined_df.index.get_level_values(0).unique()
for site in site_ids:
    tmp_df = refined_df[refined_df.index.get_level_values(0) == site].copy()
    tmp_df = tmp_df.reset_index(level=0, drop=False)
    tmp_df = tmp_df.asfreq('h')
    tmp_df['site_id'] = site
    tmp_df = tmp_df.replace(0, np.nan)
    tmp_df = tmp_df.fillna(method='ffill', limit=2)
    tmp_df = tmp_df.fillna(method='bfill', limit=2)
    tmp_df = tmp_df.interpolate(method='slinear')
    tmp_df = tmp_df.reset_index()
    tmp_df = tmp_df.set_index(['site_id','time'])
    tmp_df = tmp_df.round(0)
    new_refined_df = new_refined_df.append(tmp_df)
    print("For site {} there are {} nulls".format(site, tmp_df.isnull().sum()))

new_refined_df

Currently processing file 
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/refined_data/30.csv
Currently processing file 
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/refined_data/9.csv
Currently processing file 
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/refined_data/32.csv
Currently processing file 
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/refined_data/11.csv
Currently processing file 
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/refined_data/40.csv
Currently processing file 
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/refined_data/28.csv
Currently processing file 
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/refined_data/49.csv
Currently processing file 
/

For site 39 there are CO                0
NO2               0
PM25              0
AQI_h             0
AQI_h_Polutant    5
AQI_h_I           0
AQI_h_label       5
dtype: int64 nulls
For site 14 there are CO                0
NO2               0
PM25              0
AQI_h             0
AQI_h_Polutant    5
AQI_h_I           0
AQI_h_label       5
dtype: int64 nulls
For site 47 there are CO                0
NO2               0
PM25              0
AQI_h             0
AQI_h_Polutant    4
AQI_h_I           0
AQI_h_label       4
dtype: int64 nulls
For site 33 there are CO                0
NO2               0
PM25              0
AQI_h             0
AQI_h_Polutant    5
AQI_h_I           0
AQI_h_label       5
dtype: int64 nulls
For site 27 there are CO                0
NO2               0
PM25              0
AQI_h             0
AQI_h_Polutant    5
AQI_h_I           0
AQI_h_label       5
dtype: int64 nulls
For site 41 there are CO                0
NO2               0
PM25              0
AQI_h        

CO  NO2  PM25  AQI_h AQI_h_Polutant  AQI_h_I  \
site_id time                                                                 
30      2020-12-05 00:00:00  2.0  7.0  11.0    7.0            NO2      1.0   
        2020-12-05 01:00:00  1.0  6.0  11.0   11.0           PM25      1.0   
        2020-12-05 02:00:00  1.0  6.0  13.0   13.0           PM25      1.0   
        2020-12-05 03:00:00  1.0  6.0  18.0   18.0           PM25      1.0   
        2020-12-05 04:00:00  1.0  7.0  17.0   17.0           PM25      1.0   
...                          ...  ...   ...    ...            ...      ...   
36      2021-04-05 06:00:00  2.0  3.0  63.0   63.0           PM25      2.0   
        2021-04-05 07:00:00  3.0  3.0  64.0   64.0           PM25      2.0   
        2021-04-05 08:00:00  5.0  3.0  67.0   67.0           PM25      2.0   
        2021-04-05 09:00:00  5.0  5.0  57.0   57.0           PM25      2.0   
        2021-04-05 10:00:00  5.0  6.0  60.0   60.0           PM25      2.0   

                            AQI_h_label  
site_id time                             
30      2020-12-05 00:00:00        Good  
        2020-12-05 01:00:00        Good  
        2020-12-05 02:00:00        Good  
        2020-12-05 03:00:00        Good  
        2020-12-05 04:00:00        Good  
...                                 ...  
36      2021-04-05 06:00:00    Moderate  
        2021-04-05 07:00:00    Moderate  
        2021-04-05 08:00:00    Moderate  
        2021-04-05 09:00:00    Moderate  
        2021-04-05 10:00:00    Moderate  

[98151 rows x 7 columns]

In [28]:
site_ids = raw_df.index.get_level_values(0).unique()
for site in site_ids:
    tmp_df = raw_df[raw_df.index.get_level_values(0) == site].copy()
    tmp_df.to_csv(raw_data_folder+'/{}.csv'.format(site))

In [6]:
# Job: Read all the raw file (the input only has: site_id, time, PM25, CO, NO2)
import pandas as pd
import numpy as np
import glob
import xarray as xr

idx = pd.IndexSlice

raw_df = pd.DataFrame()
for file in raw_files:
    print('Currently processing file \n{}'.format(file))
    raw_df = raw_df.append(pd.read_csv(file, parse_dates=True, index_col=['site_id', 'time']))

raw_df = raw_df.drop(index=[16,48,49], level=0, errors='ignore')
# Deal with missing data
# raw_df = raw_df.replace(0, np.nan)
# raw_df = raw_df.fillna(method='ffill', limit=2)
# raw_df = raw_df.fillna(method='bfill', limit=2)

# raw_df = raw_df[raw_df.index.get_level_values(1) < pd.to_datetime('03-08-2021')]

Currently processing file 
../data/raw/30.csv
Currently processing file 
../data/raw/9.csv
Currently processing file 
../data/raw/32.csv
Currently processing file 
../data/raw/11.csv
Currently processing file 
../data/raw/40.csv
Currently processing file 
../data/raw/28.csv
Currently processing file 
../data/raw/46.csv
Currently processing file 
../data/raw/10.csv
Currently processing file 
../data/raw/8.csv
Currently processing file 
../data/raw/25.csv
Currently processing file 
../data/raw/7.csv
Currently processing file 
../data/raw/16.csv
Currently processing file 
../data/raw/42.csv
Currently processing file 
../data/raw/44.csv
Currently processing file 
../data/raw/37.csv
Currently processing file 
../data/raw/1.csv
Currently processing file 
../data/raw/13.csv
Currently processing file 
../data/raw/31.csv
Currently processing file 
../data/raw/26.csv
Currently processing file 
../data/raw/12.csv
Currently processing file 
../data/raw/15.csv
Currently processing file 
../data/raw

In [3]:
from features import calculate_AQI

AQI = calculate_AQI.calculate_AQI_h(raw_df)

                                CO   NO2  PM25
site_id time                                  
30      2020-12-05 00:00:00  301.2  14.8   4.9
        2020-12-05 01:00:00  183.6  12.9   5.7
        2020-12-05 02:00:00  174.5  12.9   7.6
        2020-12-05 03:00:00  159.6  12.3  12.5
        2020-12-05 04:00:00  159.3  13.8   7.5
...                            ...   ...   ...
36      2021-03-19 04:00:00  136.3   2.6  30.2
        2021-03-19 05:00:00  147.1   2.9  28.4
        2021-03-19 06:00:00  195.5   3.2  22.6
        2021-03-19 07:00:00  280.3   9.5  21.5
        2021-03-19 08:00:00  712.9   6.7  22.7

[84584 rows x 3 columns]
Done calculating NowCast for site 30
Done calculating NowCast for site 9
Done calculating NowCast for site 32
Done calculating NowCast for site 11
Done calculating NowCast for site 40
Done calculating NowCast for site 28
Done calculating NowCast for site 46
Done calculating NowCast for site 10
Done calculating NowCast for site 8
Done calculating NowCast for sit

64.06694335937499
60.33150634765625
52.214587402343746
41.40614013671875
35.40170898437501
31.349060058593754
28.721594238281256
21.80662841796875
17.549218749999994
16.770410156250005
15.530920410156252
14.662243652343752
13.72738037109375
13.4095703125
14.251330566406251
16.4229736328125
22.559619140625
28.728015136718753
35.31234130859374
38.55457763671875
39.67637939453125
37.8873779296875
35.74271240234375
35.570483398437496
36.0343994140625
33.86641845703126
33.28240966796875
35.890283203125
34.897280897394346
35.39104729625299
34.97139937557448
34.761268570644496
35.2285029055488
34.84860307029513
33.97770736274494
33.373079032594106
32.37267777521415
31.895978569606083
32.490222682197725
33.94117696441046
35.67022030765884
38.64317772124719
42.539323487704536
51.75720917255722
55.07450475636592
52.99164424807384
52.49779191582657
51.45188562572416
56.07025146484376
65.38331298828125
64.789794921875
58.86129890686883
58.7630076798069
57.40859469873725
53.953763148272884
51.49100

42.33259591693255
44.08474358955773
46.64328315428761
46.08350605109072
45.14345299880802
44.04904640473657
48.157968416731926
49.02847490413871
47.79632157389122
49.8539379745185
53.5209816014871
55.507428768021946
61.297095631151414
61.58405442754191
57.27759924663159
56.615819057935596
57.804735337732176
58.44564113544486
59.84071789492568
61.502227837407574
60.5847358218667
58.977336538750706
57.46978963830768
55.44896260799254
54.48820847269309
54.58852221770604
49.91337965197487
48.77715043852805
45.46701283733188
43.03781405363745
38.867030451839
35.03415527343749
37.509168115894504
38.22623136514185
39.6233405002812
41.30307005163581
39.2313786323153
37.353096424091795
40.46918560401391
39.34768382815865
37.96113010814849
35.93359636086166
32.06398708878646
31.22878237508664
32.146032867612384
36.648044426777325
45.19994678300253
44.37560590904909
45.600262463846846
47.39583442612335
50.601020086667795
55.09558105468751
64.69553222656249
71.49562988281251
64.69588623046876
51.5

28.718505859374996
28.846834010805544
29.460774672159385
29.340567065353106
29.54680333026965
31.611002130947607
32.28589271739934
32.68507803781518
34.86969037064801
36.41479563270566
33.80311427251814
32.91235224128771
34.213719277610856
36.61052377060726
37.36614336887412
37.42455190807434
37.918015412674336
37.57458419254988
38.33216087386948
41.124654902452434
45.4344676926672
52.14063720703125
49.2178955078125
43.88946922167808
42.848076876199535
39.76888903030998
37.27895278322356
35.36018687382398
37.744902465523126
45.34649190495464
44.510823498823044
39.59888059146994
34.161035302172614
31.30697021484375
27.04993896484375
25.9721435546875
24.833752441406247
22.56444091796875
29.180065917968747
36.887951660156254
43.09195556640625
48.293481445312494
49.493359375000004
57.04403076171875
58.7699951171875
64.28321533203125
70.08986816406251
76.69354248046874
77.39525146484375
81.39617919921878
79.89685058593749
75.16915434864816
71.43147522980638
66.21340119262187
61.820545150173

59.44682036212322
60.838672842344536
60.824348465771585
62.90707140230694
63.02352872504709
63.477192153664795
63.82968423186467
61.78618050211452
57.68022840691411
53.27833378207225
48.591334850335734
49.260365244484106
53.52529472258914
51.43018924635952
53.22380292275118
55.83868843297524
59.85337133440597
61.2755361732505
58.836124567656206
56.020643122568266
57.81061626494284
57.3031539632638
60.76174936454645
60.35286736698325
60.66190692745058
63.866638874105355
64.40934256262838
62.89652470038159
61.14161161976605
59.64150328603294
57.56818166409275
55.674082664852044
54.6635505929317
56.21645982786097
58.78864572492278
66.63822367082318
77.0937831520831
87.54514691927086
101.31634521484375
96.45407714843749
90.22344970703125
80.70827636718751
84.300732421875
84.54718017578125
89.1704833984375
97.25134971869441
101.86909704180003
101.73212578188144
105.93233324092908
119.85459803157809
120.72043159533747
114.44594984316667
103.25192653352063
89.81019173685068
83.01836395090477


...

56.80910378098236
56.257027789410444
59.604854668613434
68.72048203562933
73.10966162120788
76.90242736415416
79.59250421962636
81.9205905517839
94.24178265037132
91.09447735619271
84.019788440136
82.51883370610422
86.41363722218742
82.92521676473149
80.00844595548415
77.09704143789585
73.80023406300779
70.46069129735555
72.08652123674851
68.80718645851776
62.340860726357434
51.69018554687499
44.485766601562496
45.2830078125
42.7804443359375
42.73056640625
48.1562255859375
49.31953125
43.701416015625
46.7507080078125
42.27133789062499
43.363100343060516
42.899677418769194
42.870855376837234
43.309596104514426
44.6129355478345
44.36598796478607
46.415151602292816
48.210186288293016
48.06055004908172
45.150269051186946
43.129346069000114
36.80798654739971
31.824755729586478
30.761824999135637
28.257067871093746
27.42591552734375
28.110253906250005
27.6522216796875
25.3734375
23.183593749999996
22.8885986328125
23.743927573521578
28.89638165959843
34.89667364382559
35.52967982689252
37.84

32.7814568482697
26.622351074218752
24.8088134765625
24.25172119140625
27.874023437500004
35.684411621093744
42.2904052734375
44.89339599609376
40.1449951171875
40.32059326171875
41.80889892578125
45.452124023437506
49.62359619140624
53.36002197265625
53.578759765625
54.98797607421875
57.789075086171444
59.819377617634416
61.32278999997435
63.93737610656991
67.39682821834188
67.25442879572522
68.6563590930257
66.99936102120208
67.5730536591078
67.16519298839437
66.96202828305437
66.43489326931262
65.45285961056273
63.00455909999352
59.57858055712827
54.27687506596072
48.4551635364716
39.55380859374999
34.822741699218746
32.60701904296875
41.19951171875
53.295434570312494
61.79371337890625
64.692822265625
67.19248046874999
67.592578125
70.94288330078125
76.31824951171875
83.55626220703125
88.8255615234375
90.81080322265625
91.603564453125
85.71761067728055
83.51535322204342
80.61862938685825
77.48435348246593
72.49154970686754
63.7768743569588
56.38644513540394
56.626716369797286
55.289

27.108264160156256
26.702307128906252
29.298669433593744
33.99667968750001
38.04959848113676
45.670080566406234
52.433190917968766
60.76480712890625
69.28002929687499
68.73735351562502
65.0661376953125
71.53114013671872
80.06403808593751
88.83057861328126
90.06368408203123
87.97989501953124
84.93758544921877
82.88087592300738
83.96288101758536
82.92164481397458
78.94775537902449
76.21688845740373
75.54857395608698
75.00236446765098
73.23744691751489
68.66996508973523
65.83873746513952
56.351675185184355
55.14213472953022
58.06631460351689
60.8019215336369
67.10430892831873
71.3500630898331
75.30010315243524
77.85160307390042
83.70686362307772
87.31765322688192
90.88599853515625
93.63918457031247
97.56593017578125
98.09260773024334
97.25833255305012
96.40441423232541
91.72711694388944
86.4993641278258
77.41269791115604
68.46476818695943
57.45626220703126
48.9226806640625
46.0557861328125
48.972143554687506
56.680187988281254
70.63389892578125
79.51090087890624
69.74949951171875
69.96893

50.52557373046875
49.91046142578124
53.70235595703126
59.7493896484375
60.82304687499999
55.45965576171875
50.87799072265625
46.06200639760824
41.46197031052627
39.854127622115435
38.85146895528146
35.979671841802784
35.24729973679226
38.13785946370803
45.28124683156521
61.432666015624996
69.61231689453125
75.30238037109373
82.44813232421873
93.771240234375
92.23325195312502
89.06437988281249
81.47987060546873
80.38763427734376
89.24178466796876
99.36878662109375
94.63188476562499
82.05977279379402
65.97663574218751
57.6835693359375
49.63684082031251
40.912951660156246
37.80006103515624
38.14449462890626
35.367004394531264
33.628991699218744
32.809655761718744
33.69884033203125
37.542736816406254
37.96588134765625
35.777663395733065
36.07679418318229
37.010715365161055
51.692529296875016
60.744299316406256
60.37003173828124
67.38266601562503
74.58934326171875
79.74272460937499
88.1694091796875
87.3825927734375
85.98876953125
82.29204101562499
74.89399414062498
58.89476318359375
46.3949

35.942614746093746
36.567297363281256
28.879748535156253
25.186962890625004
23.089868164062494
24.69200439453125
32.72975296444334
32.47028105583368
34.00155403526052
42.533666992187506
42.665173339843754
49.030725097656244
55.21337890624999
51.60419921875
45.1998291015625
41.59862060546876
40.097998046875
39.54771728515624
39.561413217720535
36.10715267617484
29.089843750000007
24.6427490234375
22.168261718749996
20.481518554687497
18.7373779296875
19.81494140625
22.004541015625
26.74990234375
30.072631835937493
33.233959960937504
37.464599609375
36.329895019531236
34.912976074218754
34.95512695312502
37.27633056640625
28.786962890625002
25.242333984374994
23.12012939453125
21.158789062500006
17.87791748046875
16.387036132812497
14.291479492187497
12.393518066406251
11.694213867187502
12.794958496093747
20.695434570312507
28.145581054687508
31.620373535156247
38.50894775390626
38.65314941406251
40.27529296874999
46.736474609374994
51.867346191406256
50.332763671875
48.46563720703125
5

In [4]:
# Now we want to calculate each file AQI then write them to seperate files or just take the whole, split them by site_ids then save to refined folder.
# Job: the output only has: site_id, time, CO, NO2, PM25, AQI_h, AQI_h_Polutant, AQI_h_I
AQI = AQI[["CO", "NO2", "PM25", "AQI_h", "AQI_h_Polutant", "AQI_h_I", "AQI_h_label"]]
# AQI['site_id'] = 
site_ids = list(AQI.index.get_level_values(0).unique())
for site_id in site_ids:
    tmp = AQI.copy()
    tmp = tmp.reset_index()
    tmp = tmp[tmp['site_id'] == site_id]
    print(tmp)
#     tmp.insert(0, 'site_id', site_id)
    tmp.to_csv(refined_data_folder+"/{}.csv".format(site_id), index=False)

      site_id                time    CO   NO2  PM25  AQI_h AQI_h_Polutant  \
0           1 2020-12-05 00:00:00   2.0   2.0   0.0    2.0             CO   
1           1 2020-12-05 01:00:00   2.0   2.0  66.0   66.0           PM25   
2           1 2020-12-05 02:00:00   2.0   2.0  59.0   59.0           PM25   
3           1 2020-12-05 03:00:00   1.0   1.0  50.0   50.0           PM25   
4           1 2020-12-05 04:00:00   2.0   1.0  54.0   54.0           PM25   
...       ...                 ...   ...   ...   ...    ...            ...   
2499        1 2021-03-19 04:00:00   3.0   7.0  71.0   71.0           PM25   
2500        1 2021-03-19 05:00:00   3.0   7.0  71.0   71.0           PM25   
2501        1 2021-03-19 06:00:00   4.0   8.0  70.0   70.0           PM25   
2502        1 2021-03-19 07:00:00   5.0   9.0  68.0   68.0           PM25   
2503        1 2021-03-19 08:00:00  12.0  11.0  66.0   66.0           PM25   

      AQI_h_I AQI_h_label  
0           1        Good  
1           2    Mo

       site_id                time   CO  NO2  PM25  AQI_h AQI_h_Polutant  \
32001       27 2020-12-05 00:00:00  2.0  8.0   0.0    8.0            NO2   
32002       27 2020-12-05 01:00:00  2.0  8.0   8.0    8.0            NO2   
32003       27 2020-12-05 02:00:00  2.0  8.0   7.0    8.0            NO2   
32004       27 2020-12-05 03:00:00  2.0  8.0   6.0    8.0            NO2   
32005       27 2020-12-05 04:00:00  2.0  9.0   7.0    9.0            NO2   
...        ...                 ...  ...  ...   ...    ...            ...   
34500       27 2021-03-19 04:00:00  2.0  6.0  57.0   57.0           PM25   
34501       27 2021-03-19 05:00:00  1.0  5.0  53.0   53.0           PM25   
34502       27 2021-03-19 06:00:00  2.0  6.0  50.0   50.0           PM25   
34503       27 2021-03-19 07:00:00  2.0  6.0  48.0   48.0           PM25   
34504       27 2021-03-19 08:00:00  3.0  6.0  39.0   39.0           PM25   

       AQI_h_I AQI_h_label  
32001        1        Good  
32002        1        Good  


       site_id                time   CO   NO2  PM25  AQI_h AQI_h_Polutant  \
52033       35 2020-12-05 00:00:00  1.0   8.0   0.0    8.0            NO2   
52034       35 2020-12-05 01:00:00  1.0   7.0   7.0    7.0            NO2   
52035       35 2020-12-05 02:00:00  1.0   9.0   8.0    9.0            NO2   
52036       35 2020-12-05 03:00:00  1.0   9.0   8.0    9.0            NO2   
52037       35 2020-12-05 04:00:00  1.0   7.0   7.0    7.0            NO2   
...        ...                 ...  ...   ...   ...    ...            ...   
54532       35 2021-03-19 04:00:00  0.0   7.0  25.0   25.0           PM25   
54533       35 2021-03-19 05:00:00  0.0   5.0  20.0   20.0           PM25   
54534       35 2021-03-19 06:00:00  0.0   5.0  17.0   17.0           PM25   
54535       35 2021-03-19 07:00:00  1.0   6.0  15.0   15.0           PM25   
54536       35 2021-03-19 08:00:00  3.0  10.0  15.0   15.0           PM25   

       AQI_h_I AQI_h_label  
52033        1        Good  
52034        1   

       site_id                time   CO   NO2  PM25  AQI_h AQI_h_Polutant  \
69561       42 2020-12-05 00:00:00  2.0   6.0   0.0    6.0            NO2   
69562       42 2020-12-05 01:00:00  2.0   6.0   6.0    6.0            NO2   
69563       42 2020-12-05 02:00:00  2.0   6.0   8.0    8.0           PM25   
69564       42 2020-12-05 03:00:00  2.0   6.0   7.0    7.0           PM25   
69565       42 2020-12-05 04:00:00  2.0   7.0   9.0    9.0           PM25   
...        ...                 ...  ...   ...   ...    ...            ...   
72060       42 2021-03-19 04:00:00  1.0   7.0  43.0   43.0           PM25   
72061       42 2021-03-19 05:00:00  1.0   7.0  45.0   45.0           PM25   
72062       42 2021-03-19 06:00:00  1.0   8.0  41.0   41.0           PM25   
72063       42 2021-03-19 07:00:00  2.0   9.0  38.0   38.0           PM25   
72064       42 2021-03-19 08:00:00  3.0  10.0  32.0   32.0           PM25   

       AQI_h_I AQI_h_label  
69561        1        Good  
69562        1   

In [ ]:
def create_refined_data_files(sample_df):
    AQI = calculate_AQI_h(sample_df)
    AQI.to_csv("./lala.csv")
    return AQI
create_refined_data_files(sample_df)

In [6]:
# thudohanoi_df = thudohanoi_df.rename(columns={"PM2.5": "PM25"})
def fix_PM25_column_name(file_path):
    df = pd.read_csv(file_path.loc[0])
    df = df.rename(columns={"PM2.5": "PM25"})
    df.to_csv(file_path.loc[0], index=False)
    
all_files.apply(lambda file_path: fix_PM25_column_name(file_path), axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
dtype: object

In [43]:
def fix_time_column(file_path):
    df = pd.read_csv(file_path.loc[0])
    df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d %H:%M:%S')
    df.to_csv(file_path.loc[0], index=False)
    
all_files.apply(lambda file_path: fix_time_column(file_path), axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
dtype: object

In [33]:
def process_data(file_path):
    print(type(file_path.loc[0]))
    print(file_path.loc[0])
    df = pd.read_csv(file_path.loc[0])
    print(type(df['time']))
#     df = pd.pivot_table(df, values = 'value', index = ['site_id', 'time'], columns=['parameter'],
#               aggfunc='sum')
#     df.to_csv(file_path.loc[0])
    
all_files.apply(lambda file_path: process_data(file_path), axis=1)


<class 'str'>
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/30.csv
<class 'pandas.core.series.Series'>
<class 'str'>
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/9.csv
<class 'pandas.core.series.Series'>
<class 'str'>
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/32.csv
<class 'pandas.core.series.Series'>
<class 'str'>
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/11.csv
<class 'pandas.core.series.Series'>
<class 'str'>
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/40.csv
<class 'pandas.core.series.Series'>
<class 'str'>
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/28.csv
<class 'pandas.core.series.Series'>
<class 'str'>
/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi/4

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
dtype: object

In [17]:
# Job: built an AQI calculator and record it in every site data
# Job: Put AQI calculator in here
# Job: Read data
# Job: Calculate AQI
# Job: Put result in the same dataframe
# Job: Save result to CSV file